In [2]:
from dsm import datasets
from dsm.dsm_api import DeepSurvivalMachines
import numpy as np
from sklearn.model_selection import train_test_split
from sksurv.metrics import concordance_index_ipcw, brier_score,cumulative_dynamic_auc

In [3]:
x,t,e=datasets.load_dataset('SUPPORT')

In [4]:
times = np.quantile(t[e==1], [0.25, 0.5, 0.75]).tolist()

In [5]:
times

[14.0, 58.0, 252.0]

In [6]:
print(x.shape)
print(t.shape)
print(e.shape)

(9105, 44)
(9105,)
(9105,)


In [7]:
train_ratio = 0.70
validation_ratio = 0.10
test_ratio = 0.20
x_train, x_test, t_train, t_test, e_train, e_test = train_test_split(x, t, e, test_size=1 - train_ratio)
x_val, x_test, t_val, t_test, e_val, e_test =  train_test_split(x_test, t_test,e_test, test_size=test_ratio/(test_ratio + validation_ratio)) 

In [8]:
print(x_train.shape, t_train.shape, e_train.shape)
print(x_val.shape,t_val.shape,e_val.shape)
print(x_test.shape,t_test.shape,e_test.shape)

(6373, 44) (6373,) (6373,)
(910, 44) (910,) (910,)
(1822, 44) (1822,) (1822,)


In [9]:
model=DeepSurvivalMachines()

In [10]:
model.fit(x_train, t_train, e_train)

  0%|          | 0/10000 [00:00<?, ?it/s]/home/kishan/Documents/Projects/deepsurvival/deep_survival/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(
100%|██████████| 1/1 [00:00<00:00,  3.23it/s]


In [11]:
out_risk = model.predict_risk(x_test, times)
out_survival = model.predict_survival(x_test, times)

In [12]:
et_train = np.array([(e_train[i], t_train[i]) for i in range(len(e_train))],
                 dtype=[('e', bool), ('t', int)])
et_test = np.array([(e_test[i], t_test[i]) for i in range(len(e_test))],
                 dtype=[('e', bool), ('t', int)])

In [13]:
cis = []
brs = []
for i in range(len(times)):
    cis.append(concordance_index_ipcw(et_train, et_test, out_risk[:,i], times[i])[0])

brs.append(brier_score(et_train, et_test, out_survival, times )[1])

In [14]:
print ("Concordance Index:", cis)
print ("Brier Score:", np.mean(brs,axis=0))

Concordance Index: [0.6278103415590658, 0.5968131489314651, 0.5916086542438471]
Brier Score: [0.14186959 0.21980717 0.24599705]


In [15]:
cdauc=[]
for i in range(len(times)):
    cdauc.append(cumulative_dynamic_auc(et_train, et_test, out_risk[:,i], times[i])[0])
print ("Cumulative_dynamic AUC ", cdauc)

Cumulative_dynamic AUC  [array([0.64499677]), array([0.60635433]), array([0.60467868])]
